In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

# recursively read all "{}_output.csv" files in the current directory with their {names} in a separate column
# and return a pandas dataframe with all the data
def get_all_csv_data():
    # get all the csv files in the current directory
    # csv_files = glob.glob("*_output.csv")
    # get all csv files in the current directory and all subdirectories named '{}_output.csv'
    csv_files = glob.glob("**/*_output.csv", recursive=True)    
    # create a list of dataframes
    df_list = []
    # iterate over the csv files
    for csv_file in csv_files:
        # get the name of the file
        name = csv_file.split("_")[0].split("\\")[-1]
        # read the csv file into a pandas dataframe
        df = pd.read_csv(csv_file)
        # add a column with the name of the file
        df["name"] = name
        # append the dataframe to the list of dataframes
        df_list.append(df)
    # concatenate all the dataframes into a single dataframe
    df = pd.concat(df_list)
    # return the dataframe
    return df

data = get_all_csv_data()
data

,month and week,max loss,avg loss,min loss,std loss,min gain,avg gain,max gain,std gain,month,week,num years,avg gain/loss ratio,min gain/loss ratio,max gain/loss ratio,name
0,10-1,-46.022727,-27.470456,-8.737864,13.469174,1.724138,8.507551,19.085487,6.059706,10.0,1.0,6,-18.962904,-44.298589,10.347623,ACB
1,10-2,-33.582090,-16.124988,-4.558405,11.709362,8.632665,19.131431,38.888889,11.158847,10.0,2.0,6,3.006443,-24.949424,34.330484,ACB
2,11-1,-58.834525,-27.031342,-8.108108,17.954812,6.036745,32.925320,106.026558,37.262935,11.0,1.0,6,5.893978,-52.797780,97.918450,ACB
3,11-2,-111.440678,-42.853839,-8.890331,37.860351,2.962963,13.012459,24.352941,8.642452,11.0,2.0,6,-29.841380,-108.477715,15.462610,ACB
4,11-3,-32.962963,-18.523832,-8.704750,11.034203,1.250000,18.122671,56.100982,20.283865,11.0,3.0,6,-0.401161,-31.712963,47.396232,ACB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43,3-2,-15.077713,-15.077713,-15.077713,NaN,0.778104,0.778104,0.778104,NaN,3.0,2.0,2,-14.299609,-14.299609,-14.299609,WEED
44,3-3,-11.840436,-11.840436,-11.840436,NaN,3.488077,3.488077,3.488077,NaN,3.0,3.0,2,-8.352359,-8.352359,-8.352359,WEED
45,3-4,-15.205822,-15.205822,-15.205822,NaN,6.736179,6.736179,6.736179,NaN,3.0,4.0,2,-8.469643,-8.469643,-8.469643,WEED
46,4-3,-10.232309,-10.232309,-10.232309,NaN,15.584669,15.584669,15.584669,NaN,4.0,3.0,2,5.352360,5.352360,5.352360,WEED


In [8]:
# Create a new dataframe with the unique "month and weeks" as the columns and "top performer" as row
df = pd.DataFrame(columns=["month and week"])
df = df.set_index("month and week")
# add ['top performer 1', 'top performer 2', 'top performer 3'] as columns
df['top performer 1'] = None
df['top performer 2'] = None
df['top performer 3'] = None
# set top performer gain/loss ratio as additional columns for each top performer
df['top performer 1 gain/loss ratio'] = None
df['top performer 2 gain/loss ratio'] = None
df['top performer 3 gain/loss ratio'] = None

# iterate over the unique "month and weeks"
for month_and_week in data["month and week"].unique():
    # get the data for the current "month and week"
    df_month_and_week = data[data["month and week"] == month_and_week]
    # set num years, month, week
    df.loc[month_and_week, 'num years'] = df_month_and_week.iloc[0]["num years"]
    df.loc[month_and_week, 'month'] = df_month_and_week.iloc[0]["month"]
    df.loc[month_and_week, 'week'] = df_month_and_week.iloc[0]["week"]
    # get the top 3 performers for the current "month and week"
    top_performers = df_month_and_week.sort_values(by="avg gain/loss ratio", ascending=False).head(3)
    # set the top performers in the dataframe
    df.loc[month_and_week, 'top performer 1'] = top_performers.iloc[0]["name"]
    df.loc[month_and_week, 'top performer 2'] = top_performers.iloc[1]["name"]
    df.loc[month_and_week, 'top performer 3'] = top_performers.iloc[2]["name"]
    # set the gain/loss ratio for each top performer
    df.loc[month_and_week, 'top performer 1 gain/loss ratio'] = top_performers.iloc[0]["avg gain/loss ratio"]
    df.loc[month_and_week, 'top performer 2 gain/loss ratio'] = top_performers.iloc[1]["avg gain/loss ratio"]
    df.loc[month_and_week, 'top performer 3 gain/loss ratio'] = top_performers.iloc[2]["avg gain/loss ratio"]
    
# sort by month and then week
df = df.sort_values(by=["month", "week"], ascending=True)
# open a csv 
# save the dataframe to a csv file
df.to_csv(r"D:\repo\Stock\Seasonal-Stock\dataset\seasonal_top_performer.csv")